# Word Embeddings
Word embeddings are key in NLP and are an example of representation learning, the core of all machine learning.

Study notes on the pytorch article on word embeddings: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html

$p(w_i|w_{i-1},w_{i-2},...,w_{i-n+1})$

In [7]:
CONTEXT_SIZE = 2 # Words given as context for the predicted word
EMBEDDING_DIM = 10 # The size of the word embedding vector

# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()

# we should tokenize the input, but we will ignore that for now

# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]

# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [12]:
import torch.optim as optim
from tqdm import trange

losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


for epoch in trange(100, desc="Training word embeddings", unit="Epochs (One cycle through all training data)"):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

Training word embeddings: 100%|██████████| 100/100 [00:06<00:00, 15.97Epochs (One cycle through all training data)/s]

[522.3798503875732, 520.0369343757629, 517.7073473930359, 515.3919634819031, 513.0900731086731, 510.7990527153015, 508.5183641910553, 506.2486352920532, 503.98994183540344, 501.7406349182129, 499.4989609718323, 497.2660331726074, 495.03919768333435, 492.81839084625244, 490.6032373905182, 488.3936686515808, 486.1872775554657, 483.98573207855225, 481.78607749938965, 479.5898847579956, 477.39561796188354, 475.20340037345886, 473.01307582855225, 470.8242259025574, 468.6363205909729, 466.44900846481323, 464.26203894615173, 462.0750849246979, 459.8871500492096, 457.69857835769653, 455.50883889198303, 453.3182303905487, 451.12744665145874, 448.9336881637573, 446.73854804039, 444.5394778251648, 442.33684039115906, 440.12960410118103, 437.91625118255615, 435.6982479095459, 433.4755742549896, 431.24888706207275, 429.0162236690521, 426.77768552303314, 424.53383445739746, 422.2834415435791, 420.0268088579178, 417.7637734413147, 415.4932042360306, 413.21672761440277, 410.9325864315033, 408.64015483